<a href="https://colab.research.google.com/github/sagewoodard/GEE_Satellite_Data_Preprocessing/blob/main/GeoTIFF_Merge.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Generate tiles programmatically

# Import necessary libraries
import ee
from google.colab import drive

# Authenticate and initialize GEE
ee.Authenticate()
ee.Initialize(project='satellite-data-442019')

# Mount Google Drive
drive.mount('/content/drive')

# Define region of interest and grid size
region = ee.Geometry.Rectangle([-120.0, 32.529521, -114.131211, 34.5])  # Full region
min_lon, min_lat, max_lon, max_lat = -120.0, 32.529521, -114.131211, 34.5
tile_size = 0.5  # Tile size in degrees

# Generate tiles as a list of smaller rectangles
tiles = []
current_lat = min_lat
while current_lat < max_lat:
    current_lon = min_lon
    while current_lon < max_lon:
        tile = ee.Geometry.Rectangle([
            current_lon,
            current_lat,
            min(current_lon + tile_size, max_lon),
            min(current_lat + tile_size, max_lat)
        ])
        tiles.append(tile)
        current_lon += tile_size
    current_lat += tile_size
print(f"Generated {len(tiles)} tiles.")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Generated 48 tiles.


In [ ]:
# Export each tile

# Iterate over tiles and export them
for i, tile in enumerate(tiles):
    # Filter the Sentinel-2 dataset for each tile
    dataset = ee.ImageCollection('COPERNICUS/S2_HARMONIZED') \
        .filterDate('2023-01-01', '2023-12-31') \
        .filterBounds(tile) \
        .filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE', 20))

    # Create a mosaic for the current tile
    mosaic = dataset.mosaic().clip(tile).uint16()

    # Export the tile
    export_task = ee.batch.Export.image.toDrive(
        image=mosaic,
        description=f'SoCal_Tile_{i}',
        folder='EarthEngineExports',
        fileNamePrefix=f'S2_SoCal_Coastline_Tile_{i}',
        region=tile.getInfo()['coordinates'],
        scale=30,
        maxPixels=1e13,
        fileFormat='GeoTIFF'
    )
    export_task.start()
    print(f"Started export for Tile {i}: Task ID = {export_task.id}")

Started export for Tile 0: Task ID = WYQ4HB4ZAFFJRSYXCDD4XW2H
Started export for Tile 1: Task ID = KSW3BS7B6QLW4OSF46TQBA2Z
Started export for Tile 2: Task ID = HD6ZNZLARX6WNXFAONFBNKPN
Started export for Tile 3: Task ID = LPHK5LZRIALG6MA5K3BCAEE2
Started export for Tile 4: Task ID = ZBYW52NXVUPPSXHY7D5YNZ5P
Started export for Tile 5: Task ID = IFIKB4KYLXAQJXS4BXIDOJUR
Started export for Tile 6: Task ID = FGPZMJCPRCNN6GKPVSI3BJ57
Started export for Tile 7: Task ID = WHKDKYNST76PRU5736PDJ7B3
Started export for Tile 8: Task ID = KWKZJEFEA4YLENOXFQYCD4NV
Started export for Tile 9: Task ID = R6ZD55JBP2XZPC4WBOGNA43N
Started export for Tile 10: Task ID = TRMSXCYECFKCTLVEKO5SP2TL
Started export for Tile 11: Task ID = FBQEAIBJMKHXX4G6IVFSG6X4
Started export for Tile 12: Task ID = SOLKEOO7GHUQWGJVDDG337FN
Started export for Tile 13: Task ID = RTKRE5NOA4AQ4A4O3UO34XD3
Started export for Tile 14: Task ID = 5CZ5FEKG5GK5EIP3RRIXEFBJ
Started export for Tile 15: Task ID = 6Y6FU2SPPHVO3V7RL2SCGI2J
St

In [ ]:
# Monitor task progress

# Function to monitor tasks
import time

def monitor_tasks():
    print("Monitoring GEE export tasks...")
    while True:
        tasks = ee.batch.Task.list()
        active_tasks = [task for task in tasks if task.state in ('READY', 'RUNNING')]
        for task in tasks:
            print(f"Task ID: {task.id}, Description: {task.config['description']}, State: {task.state}")
        if not active_tasks:
            print("All tasks completed!")
            break
        time.sleep(60)  # Check every minute

monitor_tasks()

Streaming output truncated to the last 5000 lines.
Task ID: HK4SNQAO5TVJ75WORCVRN5LL, Description: California_Coast_S2_SR, State: FAILED
Task ID: O2OGOOWFJJHIEB6YQQKUSEUC, Description: California_Coast_S2_SR, State: COMPLETED
Task ID: URA2S55LSVPUUJBFDNOKRESQ, Description: California_Coast_S2_SR, State: COMPLETED
Task ID: 7PL7ZGB5GUHUQNS72OB5DNR7, Description: California_Coast_S2_SR, State: CANCELLED
Task ID: E5VVRATUSAQI33JNWOUVVRZW, Description: California_Coast_S2_SR, State: FAILED
Task ID: XFJYQONRWEMPLCNJCEZO6J5W, Description: California_Coast_S2_SR, State: FAILED
Task ID: SMWIXIJ377TFULL7UCQJEYZI, Description: California_Coast_S2_SR, State: FAILED
Task ID: SMEQVZPTT2HG47APQGWSCI7U, Description: Grand_Canyon_S2_SR, State: COMPLETED
Task ID: VOX4TBVLMNJXGR6XTKH2TMQX, Description: Grand_Canyon_S2_SR, State: COMPLETED
Task ID: FCFEXAMMHAMHNU4TOFBEB3SL, Description: Grand_Canyon_S2_SR, State: COMPLETED
Task ID: V4QFKJWMITSQBOVY57SOL3X4, Description: Grand_Canyon_S2_SR, State: COMPLETE

In [ ]:
import os
import glob

# Define paths for .tif files
tif_directory = '/content/drive/My Drive/EarthEngineExports'
merged_tif_path = '/content/drive/My Drive/EarthEngineExports/merged_output.tif'

# List all .tif files in the specified directory
os.chdir(tif_directory)
tile_files = glob.glob('S2_SoCal_Coastline_Tile_*.tif')

# Verify that all expected tiles are present
if len(tile_files) == 48:
    print(f"Found all 48 tiles for merging.")
else:
    print(f"Expected 48 tiles, but found {len(tile_files)}. Please check your exports.")

# Merge the .tif files into a single file using GDAL
if tile_files:
    print("Merging tiles...")
    # Properly quote paths for GDAL compatibility
    quoted_tile_files = [f'"{os.path.join(tif_directory, file)}"' for file in tile_files]
    quoted_merged_path = f'"{merged_tif_path}"'

    # Execute gdal_merge.py with correctly quoted paths
    merge_command = f"gdal_merge.py -o {quoted_merged_path} -of GTiff -co BIGTIFF=YES -co COMPRESS=LZW {' '.join(quoted_tile_files)}"
    print(f"Running command: {merge_command}")
    os.system(merge_command)

    print(f"Merged file saved to: {merged_tif_path}")
else:
    print("No tiles found to merge.")

Found all 48 tiles for merging.
Merging tiles...
Running command: gdal_merge.py -o "/content/drive/My Drive/EarthEngineExports/merged_output.tif" -of GTiff -co BIGTIFF=YES -co COMPRESS=LZW "/content/drive/My Drive/EarthEngineExports/S2_SoCal_Coastline_Tile_0.tif" "/content/drive/My Drive/EarthEngineExports/S2_SoCal_Coastline_Tile_1.tif" "/content/drive/My Drive/EarthEngineExports/S2_SoCal_Coastline_Tile_4.tif" "/content/drive/My Drive/EarthEngineExports/S2_SoCal_Coastline_Tile_2.tif" "/content/drive/My Drive/EarthEngineExports/S2_SoCal_Coastline_Tile_5.tif" "/content/drive/My Drive/EarthEngineExports/S2_SoCal_Coastline_Tile_3.tif" "/content/drive/My Drive/EarthEngineExports/S2_SoCal_Coastline_Tile_6.tif" "/content/drive/My Drive/EarthEngineExports/S2_SoCal_Coastline_Tile_7.tif" "/content/drive/My Drive/EarthEngineExports/S2_SoCal_Coastline_Tile_8.tif" "/content/drive/My Drive/EarthEngineExports/S2_SoCal_Coastline_Tile_10.tif" "/content/drive/My Drive/EarthEngineExports/S2_SoCal_Coastli